# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 28 2023 9:59AM,258349,10,0086459064,ISDIN Corporation,Released
1,Feb 28 2023 9:59AM,258349,10,0086461580,ISDIN Corporation,Released
2,Feb 28 2023 9:59AM,258349,10,0086462071,ISDIN Corporation,Released
3,Feb 28 2023 9:59AM,258349,10,0086459143,ISDIN Corporation,Released
4,Feb 28 2023 9:59AM,258349,10,0086459150,ISDIN Corporation,Released
5,Feb 28 2023 9:59AM,258349,10,0086459155,ISDIN Corporation,Released
6,Feb 28 2023 9:59AM,258349,10,0086459156,ISDIN Corporation,Released
7,Feb 28 2023 9:59AM,258349,10,0086459446,ISDIN Corporation,Released
8,Feb 28 2023 9:59AM,258349,10,0086461561,ISDIN Corporation,Released
9,Feb 28 2023 9:59AM,258349,10,0086459079,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
44,258345,Released,2
45,258346,Released,3
46,258347,Released,18
47,258348,Released,2
48,258349,Released,15


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
258345,NaN,NaN,2.0
258346,NaN,NaN,3.0
258347,NaN,NaN,18.0
258348,NaN,NaN,2.0
258349,NaN,NaN,15.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258202,98.0,1.0,14.0
258219,0.0,0.0,61.0
258225,0.0,0.0,7.0
258227,0.0,0.0,1.0
258234,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258202,98,1,14
258219,0,0,61
258225,0,0,7
258227,0,0,1
258234,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258202,98,1,14
1,258219,0,0,61
2,258225,0,0,7
3,258227,0,0,1
4,258234,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258202,98,1,14
1,258219,,,61
2,258225,,,7
3,258227,,,1
4,258234,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 28 2023 9:59AM,258349,10,ISDIN Corporation
15,Feb 28 2023 9:54AM,258348,15,"Mizner Bioscience, LLC"
17,Feb 28 2023 9:52AM,258347,10,"Methapharm, Inc."
35,Feb 28 2023 9:47AM,258346,10,Emerginnova
38,Feb 28 2023 9:47AM,258308,15,"Virtus Pharmaceuticals, LLC"
54,Feb 28 2023 9:41AM,258345,10,Eye Pharma Inc
56,Feb 28 2023 9:40AM,258344,10,"Senseonics, Incorporated"
57,Feb 28 2023 9:39AM,258267,15,"Person & Covey, Inc."
89,Feb 28 2023 9:39AM,258343,20,Granules Consumer Health
90,Feb 28 2023 9:33AM,258260,15,"Brookfield Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 28 2023 9:59AM,258349,10,ISDIN Corporation,,,15
1,Feb 28 2023 9:54AM,258348,15,"Mizner Bioscience, LLC",,,2
2,Feb 28 2023 9:52AM,258347,10,"Methapharm, Inc.",,,18
3,Feb 28 2023 9:47AM,258346,10,Emerginnova,,,3
4,Feb 28 2023 9:47AM,258308,15,"Virtus Pharmaceuticals, LLC",,,16
5,Feb 28 2023 9:41AM,258345,10,Eye Pharma Inc,,,2
6,Feb 28 2023 9:40AM,258344,10,"Senseonics, Incorporated",,,1
7,Feb 28 2023 9:39AM,258267,15,"Person & Covey, Inc.",,,32
8,Feb 28 2023 9:39AM,258343,20,Granules Consumer Health,,,1
9,Feb 28 2023 9:33AM,258260,15,"Brookfield Pharmaceuticals, LLC",,,21


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 28 2023 9:59AM,258349,10,ISDIN Corporation,15,,
1,Feb 28 2023 9:54AM,258348,15,"Mizner Bioscience, LLC",2,,
2,Feb 28 2023 9:52AM,258347,10,"Methapharm, Inc.",18,,
3,Feb 28 2023 9:47AM,258346,10,Emerginnova,3,,
4,Feb 28 2023 9:47AM,258308,15,"Virtus Pharmaceuticals, LLC",16,,
5,Feb 28 2023 9:41AM,258345,10,Eye Pharma Inc,2,,
6,Feb 28 2023 9:40AM,258344,10,"Senseonics, Incorporated",1,,
7,Feb 28 2023 9:39AM,258267,15,"Person & Covey, Inc.",32,,
8,Feb 28 2023 9:39AM,258343,20,Granules Consumer Health,1,,
9,Feb 28 2023 9:33AM,258260,15,"Brookfield Pharmaceuticals, LLC",21,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 28 2023 9:59AM,258349,10,ISDIN Corporation,15,,
1,Feb 28 2023 9:54AM,258348,15,"Mizner Bioscience, LLC",2,,
2,Feb 28 2023 9:52AM,258347,10,"Methapharm, Inc.",18,,
3,Feb 28 2023 9:47AM,258346,10,Emerginnova,3,,
4,Feb 28 2023 9:47AM,258308,15,"Virtus Pharmaceuticals, LLC",16,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 28 2023 9:59AM,258349,10,ISDIN Corporation,15.0,NaN,NaN
1,Feb 28 2023 9:54AM,258348,15,"Mizner Bioscience, LLC",2.0,NaN,NaN
2,Feb 28 2023 9:52AM,258347,10,"Methapharm, Inc.",18.0,NaN,NaN
3,Feb 28 2023 9:47AM,258346,10,Emerginnova,3.0,NaN,NaN
4,Feb 28 2023 9:47AM,258308,15,"Virtus Pharmaceuticals, LLC",16.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 28 2023 9:59AM,258349,10,ISDIN Corporation,15.0,0.0,0.0
1,Feb 28 2023 9:54AM,258348,15,"Mizner Bioscience, LLC",2.0,0.0,0.0
2,Feb 28 2023 9:52AM,258347,10,"Methapharm, Inc.",18.0,0.0,0.0
3,Feb 28 2023 9:47AM,258346,10,Emerginnova,3.0,0.0,0.0
4,Feb 28 2023 9:47AM,258308,15,"Virtus Pharmaceuticals, LLC",16.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4907868,341.0,9.0,0.0
12,1033253,25.0,2.0,0.0
15,1807861,112.0,1.0,98.0
16,258329,0.0,1.0,0.0
19,1549933,13.0,18.0,0.0
20,516627,1.0,5.0,18.0
21,516558,1.0,1.0,0.0
22,258338,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4907868,341.0,9.0,0.0
1,12,1033253,25.0,2.0,0.0
2,15,1807861,112.0,1.0,98.0
3,16,258329,0.0,1.0,0.0
4,19,1549933,13.0,18.0,0.0
5,20,516627,1.0,5.0,18.0
6,21,516558,1.0,1.0,0.0
7,22,258338,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,341.0,9.0,0.0
1,12,25.0,2.0,0.0
2,15,112.0,1.0,98.0
3,16,0.0,1.0,0.0
4,19,13.0,18.0,0.0
5,20,1.0,5.0,18.0
6,21,1.0,1.0,0.0
7,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,341.0
1,12,Released,25.0
2,15,Released,112.0
3,16,Released,0.0
4,19,Released,13.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,0.0,0.0,98.0,0.0,0.0,18.0,0.0,0.0
Executing,9.0,2.0,1.0,1.0,18.0,5.0,1.0,1.0
Released,341.0,25.0,112.0,0.0,13.0,1.0,1.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,0.0,0.0,98.0,0.0,0.0,18.0,0.0,0.0
1,Executing,9.0,2.0,1.0,1.0,18.0,5.0,1.0,1.0
2,Released,341.0,25.0,112.0,0.0,13.0,1.0,1.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,0.0,0.0,98.0,0.0,0.0,18.0,0.0,0.0
1,Executing,9.0,2.0,1.0,1.0,18.0,5.0,1.0,1.0
2,Released,341.0,25.0,112.0,0.0,13.0,1.0,1.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()